# Machine Vision - Assignment 2: Gradient Descent and Neural Networks in PyTorch

---

Prof. Dr. Markus Enzweiler, Esslingen University of Applied Sciences

markus.enzweiler@hs-esslingen.de

---

### This is the second assignment for the "Machine Vision" lecture.
It covers:
* polynomial regression using gradient descent
* getting started with [PyTorch](https://pytorch.org)
* training multilayer perceptrons for traffic sign recognition
* working with public benchmark datasets ([German Traffic Sign Recognition Benchmark](https://benchmark.ini.rub.de/gtsrb_news.html))

**Make sure that "GPU" is selected in Runtime -> Change runtime type**

To successfully complete this assignment, it is assumed that you already have some experience in Python and numpy. You can either use [Google Colab](https://colab.research.google.com/) for free with a private (dedicated) Google account (recommended) or a local Jupyter installation.

---


## Exercise 1 - PyTorch Tutorial (10 points)


### Introduction to PyTorch

Work through the ["Introduction to PyTorch" tutorial](https://pytorch.org/tutorials/beginner/basics/intro.html) consisting of nine topics:

*   Learn the Basics
*   Quickstart
*   Tensors
*   Datasets & DataLoaders
*   Transforms
*   Build Model
*   Autograd
*   Optimization
*   Save & Load Model

You can run each part in Colab and inspect / modify the code and data. There are certainly some details that you do not yet understand. Don't let that stop you, but try to get a good overall view on working with PyTorch.

**Please answer the questions below (directly in the notebook):**

### Question 1 (3 points)

 Why does PyTorch have a dedicated tensor data structure (`torch.tensor`) and does not use NumPy multidimensional arrays exclusively?

### Your Answer:
Tensors can run on GPU or other hardware accelarators, which NumPy arrays can not






### Question 2 (4 points)

 In the ["Build Model"](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) section of the tutorial, there is a definition of a neural network, as follows:


```
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
```

Explain the different parameters of the `nn.Linear()` layers. How are the numeric values, e.g. `28*28`, `512`, `10`, determined?  

### Your Answer:

28*28 = Image Size of the dataset - so from each pixel one input

10 = output of the neural network, dataset has 10 labels. So for each label we have 1 output

512 = choose of hidden layers. Can be determined by the user. Its common to use numbers by the power of 2. You can test with which number the model performs best






### Question 3 (3 points)

 In the ["Optimization"](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) section of the tutorial, an example of a training loop is given:

 ```
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
 ```

 Explain the effect of the function calls `optimizer.zero_grad()` and `optimizer.step()`.

### Your Answer:

optimizer.zero_grad() = reset the gradients, so we have no doulbe counting

optimizer.step() = adjust the parameters by the collected gradients





## Preparations and Imports


### Package Path

In [ ]:
# Package Path (this needs to be adapted)
packagePath = "./" # local
if 'google.colab' in str(get_ipython()):
  packagePath = "/content/drive/My Drive/MachineVis2/template"   # Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import important libraries (you should probably start with these lines all the time ...)

In [ ]:
# os, glob, time, logging
import os, glob, time, logging

# NumPy
import numpy as np

# OpenCV
import cv2

# Matplotlib
import matplotlib.pyplot as plt
# make sure we show all plots directly below each cell
%matplotlib inline

# PyTorch
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Some Colab specific packages
if 'google.colab' in str(get_ipython()):
  # image display
  from google.colab.patches import cv2_imshow


### Some helper functions that we will need

In [ ]:
def my_imshow(image, windowTitle=None, size=20, depth=3):
  '''
  Displays an image and differentiates between Google Colab and a local Python installation.

  Args:
    image: The image to be displayed

  Returns:
    -
  '''

  if 'google.colab' in str(get_ipython()):
    print(windowTitle)
    cv2_imshow(image)
  else:
    if (size):
      (h, w) = image.shape[:2]
      aspectRatio = float(h)/w
      figsize=(size, size * aspectRatio)
      plt.figure(figsize=figsize)

    if (windowTitle):
      plt.title(windowTitle)

    if (depth == 1):
      plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    elif (depth == 3):
      plt.imshow(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    else:
      plt.imshow(image)
    plt.axis('off')
    plt.show()

### In Google Colab only:
Mount the Google Drive associated with your Google account. You will have to click the authorization link and enter the obtained authorization code here in Colab.

In [ ]:
# Mount Google Drive
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

### PyTorch Trainer and Test Class

In the project package, I have provided a Python file `torchHelpers.py` that contains two classes `Trainer` and `Tester`. These classes contain the neural network training loop and test code, similar to what you have already seen in the tutorial.

The classes can be used as follows (see the documentation of the individual classes):

```
# Train a neural network model
# create a trainer
trainer = Trainer(model, lossFunction, optimizer, device, logLevel=logging.INFO)
# train the model
trainer.train(trainLoader, valLoader, numberOfEpochs)

# Test a neural network model
# create a tester
tester = Tester(model, device, logLevel=logging.INFO)
# test the model
tester.test(testLoader)
```

Error and accuracy metrics are available after training / testing via `trainer.metrics` and `tester.metrics`.

In [ ]:
import sys
sys.path.append(packagePath)
print("If the import does not work, most likely your 'packagePath' is not set correctly!")
print(f'packagePath: {packagePath}')

from torchHelpers import Trainer, Tester

help(Trainer)

## Exercise 2 - Polynomial regression (10 points)

This exercise involves fitting a polynomial model to given data using gradient descent. It is similar to the ```linear_regression``` example from [https://github.com/menzHSE/cv-ml-lecture-notebooks](https://github.com/menzHSE/cv-ml-lecture-notebooks) that we have discussed in the lecture, but with a polynomial model instead of the linear model.

The polynomial model is given by:
$$
y = f(x) = a \cdot x^3 + b \cdot x^2 + c \cdot x + d
$$

Your task is to estimate $a$, $b$, $c$ and $d$ using gradient descent using mean squared error loss between the predictions $\hat{y_i}$ of our model and true values $y_i$:
$$
L(a, b, c, d) \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y_i})^2 = \frac{1}{N} \sum_{i=1}^{N} \left(y_i - (a \cdot x_i^3 + b \cdot x_i^2 + c \cdot x_i + d)\right)^2
$$

**Tasks**

* Derive the gradient descent update rules to estimate $a$, $b$, $c$, and $d$
* Implement gradient descent to estimate $a$, $b$, $c$, and $d$ using your derived update rules



### Load and visualize the data

In [ ]:
data_path = os.path.join(packagePath, "data", "poly.csv")

# load data from csv
data = np.loadtxt(data_path, delimiter=",")
X = torch.Tensor(data[:, 0])
Y = torch.Tensor(data[:, 1])
# Visualize
plt.scatter(X, Y, alpha=0.5)
plt.title("Scatter plot of given data")
plt.show()

### Initialize parameters


In [ ]:
a = torch.randn(1, dtype=torch.float32)
b = torch.randn(1, dtype=torch.float32)
c = torch.randn(1, dtype=torch.float32)
d = torch.randn(1, dtype=torch.float32)

print(f"Initial params : {a,b,c,d}")

### Estimate $a$, $b$, $c$, $d$ using gradient descent

Use the following hyperparameters:
* 100000 iterations of gradient descent
* learning rate 1e-5

In [ ]:
def poly_model(a, b, c, d, x):
  return a * x**3 + b * x**2 + c * x + d
def loss_msl(y_pred, y):
  return torch.mean(torch.square(y-y_pred))
def loss_a(x,y,a,b,c,d):
  return torch.mean(-2 * (y - poly_model(a,b,c,d,x))*x**3)
def loss_b(x,y,a,b,c,d):
  return torch.mean(-2 * (y - poly_model(a,b,c,d,x))*x**2)
def loss_c(x,y,a,b,c,d):
  return torch.mean(-2 * (y - poly_model(a,b,c,d,x))*x)
def loss_d(x,y,a,b,c,d):
  return torch.mean(-2 * (y - poly_model(a,b,c,d,x)))


#Hyperparameters

num_iteration = 10000
learning_rate = 1e-5

for iteration in range (num_iteration):
  y_pred = poly_model(a,b,c,d,X)

  loss = loss_msl(y_pred, Y)

  grad_a = loss_a(X,Y,a,b,c,d)
  grad_b = loss_b(X,Y,a,b,c,d)
  grad_c = loss_c(X,Y,a,b,c,d)
  grad_d = loss_d(X,Y,a,b,c,d)

  with torch.no_grad():
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

  if iteration % 500 == 0 or iteration == 0:
    print(f"Iteration {iteration} | Loss {loss.item()} | a {a.item()} | b{b.item()} | c {c.item()} | d {d.item()}")

print(f"Final| a {a.item()} | b{b.item()} | c {c.item()} | d {d.item()}")

### Visualize the polynomial fit

In [ ]:
# Visualize
plt.scatter(X, Y, alpha=0.5)
plt.title("Scatter plot of fitted polynomial")

# Plot the recovered line
Y_model = poly_model(a, b, c, d, X)
plt.plot(X.tolist(), Y_model.tolist(), color="red")

plt.legend(["data", f"f(x)"])
plt.show()

## Exercise 3 - Traffic Sign Classification using Multilayer Perceptrons in PyTorch (10 points)

In this exercise you will train a multilayer perception neural network using PyTorch on the [German Traffic Sign Recognition Benchmark](https://benchmark.ini.rub.de/gtsrb_news.html) dataset. There will be no previous feature transform, i.e. the raw pixel values are the input to the neural network.

### Automatically select the best available device (**PROVIDED**)

**In Colab: Make sure that "GPU" is selected in Runtime -> Change runtime type**

You should have a GPU device available, e.g.:

```
Using device: cuda
Tesla T4
```

We will transfer our model and data to this device later on in the Assignment. PyTorch takes care of all the particular device handling automatically, i.e. we will not have to explicitly deal with CUDA / MPS.

In [ ]:
# Check the devices that we have available and prefer CUDA over MPS and CPU
def autoselectDevice(verbose=1):

    # default: CPU
    device = torch.device('cpu')

    if torch.cuda.is_available():
        # CUDA
        device = torch.device('cuda')
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        # MPS (acceleration on Apple M-series SoC)
        device = torch.device('mps')

    if verbose:
        print('Using device:', device)

    # Additional Info when using cuda
    if verbose and device.type == 'cuda':
        print(torch.cuda.get_device_name(0))

    return device

# We transfer our model and data later to this device. If this is a GPU
# PyTorch will take care of everything automatically.
device = autoselectDevice(verbose=1)


### Getting familiar with the GTSRB dataset (**PROVIDED**)

In [ ]:
# GTSRB is available as standard dataset in PyTorch. Nice :)

# Data is loaded and processed in batches of 'batchSize' images
batchSize = 24

# We can add a chain of transforms that is applied to the original data, e.g.
#    resize all images to the same dimensions, e.g. 64x64 pixels
#    convert (batch of) images to a tensor
#    normalize pixel values (to 0-1)

transform = transforms.Compose(
    [transforms.Resize((64, 64)), # resize to 64x64 pixels
     transforms.ToTensor()        # convert to tensor. This will also normalize pixels to 0-1
     ])

# We construct several DataLoaders that take care of loading, storing, caching, pre-fetching the dataset.
# We will have one DataLoader for training, validation and test data.

# Training data
trainSet = torchvision.datasets.GTSRB(root='./data', split='train',
                                      download=True, transform=transform)
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize,
                                          shuffle=True, pin_memory=True, num_workers=2)
numTrainSamples = len(trainSet)

# Validation and test data
# GTSRB only provides a single test set. To create a validation and test set,
# we split the original GTSRB test set into two parts. The validation set is
# used to tune performance during training. The test set is only used AFTER
# training to evaluation the final performance.

gtsrbTestSet = torchvision.datasets.GTSRB(root='./data', split='test',
                                          download=True, transform=transform)

# Split the original GTSRB test data into 75% used for validation and 25% used for testing
# We do not need to shuffle the data, as we are processing every validation / test image exactly once
length75Percent = int(0.75 * len(gtsrbTestSet))
length25Percent = len(gtsrbTestSet) - length75Percent
lengths = [length75Percent, length25Percent]
valSet, testSet = torch.utils.data.random_split(gtsrbTestSet, lengths)
valLoader = torch.utils.data.DataLoader(valSet, batch_size=batchSize,
                                        shuffle=False, pin_memory=True, num_workers=2)
numValSamples = len(valSet)

testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize,
                                         shuffle=False, pin_memory=True, num_workers=2)
numTestSamples = len(testSet)

# Available traffic sign classes in the dataset
classes = [
    "Speed limit (20km/h)",
    "Speed limit (30km/h)",
    "Speed limit (50km/h)",
    "Speed limit (60km/h)",
    "Speed limit (70km/h)",
    "Speed limit (80km/h)",
    "End of speed limit (80km/h)",
    "Speed limit (100km/h)",
    "Speed limit (120km/h)",
    "No passing",
    "No passing for vehicles over 3.5 metric tons",
    "Right-of-way at the next intersection",
    "Priority road",
    "Yield",
    "Stop",
    "No vehicles",
    "Vehicles over 3.5 metric tons prohibited",
    "No entry",
    "General caution",
    "Dangerous curve to the left",
    "Dangerous curve to the right",
    "Double curve",
    "Bumpy road",
    "Slippery road",
    "Road narrows on the right",
    "Road work",
    "Traffic signals",
    "Pedestrians",
    "Children crossing",
    "Bicycles crossing",
    "Beware of ice/snow",
    "Wild animals crossing",
    "End of all speed and passing limits",
    "Turn right ahead",
    "Turn left ahead",
    "Ahead only",
    "Go straight or right",
    "Go straight or left",
    "Keep right",
    "Keep left",
    "Roundabout mandatory",
    "End of no passing",
    "End of no passing by vehicles over 3.5 metric tons",
]

numClasses = len(classes)

### Print dataset statistics (**PROVIDED**)

In [ ]:
print("Dataset Statistics")
print(f"  # of training samples:   {numTrainSamples}")
print(f"  # of validation samples: {numValSamples}")
print(f"  # of test samples:       {numTestSamples}")
print(f"  # of different classes:  {numClasses}")

### Visualize the data (**PROVIDED**)

In [ ]:
# Visualize a random batch of data from the data set

def imshow(img):
    npimg = img.cpu().numpy() # make sure image is in host memory

    # normalize to 0-1 for visualization
    minPixel = np.min(npimg)
    maxPixel = np.max(npimg)
    npimg = npimg - minPixel
    npimg = npimg / (maxPixel-minPixel)

    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()


numRows = 8

# get a single random batch of training images
dataIter = iter(trainLoader)
images, labels = next(dataIter)

# print labels
for i in range( batchSize // numRows ):
    print('\n'.join(f'Image {j:2d}: {classes[labels[j]]:5s}' for j in range((i*numRows), (i*numRows)+numRows)))

# show images
imshow(torchvision.utils.make_grid(images, nrow=numRows))

### Neural Network Model Definition (**add your code here**)

We want to design a standard "feed-forward" multilayer perceptron as seen in the [tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html). In PyTorch terms, this is a Python class that subclasses [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) and contains `__init__()` and `forward()`. Note that you do not have to use `nn.Sequential` as seen in the [tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html). This only groups layers together but is not necessary. You can also treat each layer separately, e.g. as shown in this [tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolutional-neural-network).


We will need the following layers (input to output):

* The input will be the raw pixel values, i.e. 12288 values (images of 64x64x3 flattened into a single vector)
* 4 [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) layers with 512, 256, 128, 64 neurons each
* The output layer is a [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) layer with `numClasses` neurons, one neuron per class
* All neurons except for neurons in the output layer should have [`torch.nn.functional.leaky_relu`](https://pytorch.org/docs/stable/generated/torch.nn.functional.leaky_relu.html#torch.nn.functional.leaky_relu) activation functions
* **Important: The output layer must not have any activation function. It will be automatically applied in the loss computation (softmax activation for CrossEntropy loss, as seen in the lecture).**


In [ ]:
## Define the layers of the MLP network model

###### YOUR CODE GOES HERE ######
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
      super().__init__()
      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(64*64*3, 512)
      self.fc2 = nn.Linear(512, 256)
      self.fc3 = nn.Linear(256, 128)
      self.fc4 = nn.Linear(128, 64)
      self.fc5 = nn.Linear(64, numClasses)

    def forward(self, x):
      x = self.flatten(x)
      x = F.leaky_relu(self.fc1(x))
      x = F.leaky_relu(self.fc2(x))
      x = F.leaky_relu(self.fc3(x))
      x = F.leaky_relu(self.fc4(x))
      x = self.fc5(x)
      return x
#################################


### Print a summary of the structure of our network using the `torchinfo` package. (**PROVIDED**)

The result should look similar to:
```
==========================================================================================
Layer (type (var_name))                  Output Shape              Param #
==========================================================================================
Net (Net)                                [1, 43]                   --
├─Linear (fc1)                           [1, 512]                  6,291,968
├─Linear (fc2)                           [1, 256]                  131,328
├─Linear (fc3)                           [1, 128]                  32,896
├─Linear (fc4)                           [1, 64]                   8,256
├─Linear (fc5)                           [1, 43]                   2,795
==========================================================================================
Total params: 6,467,243
Trainable params: 6,467,243
Non-trainable params: 0
Total mult-adds (M): 6.47
==========================================================================================
Input size (MB): 0.05
Forward/backward pass size (MB): 0.01
Params size (MB): 25.87
Estimated Total Size (MB): 25.93
==========================================================================================
```

In [ ]:
# Instatiate our neural network
network = Net()

# Print a summary of the net
%pip install torchinfo

from torchinfo import summary
summary(network, input_size=(1, 3, 64, 64), row_settings=["var_names"])

### Question

Why is the number of learnable parameters of the first hidden layer `6,291,968`?

Vollvernetzung -> 12288 (Input) x 512 (Neuronen) = 6,291,456

Bias --> 6,291,456 + 512 Bias = 6,291,968

#### Your Answer:



### Neural Network Training (**add your code here**)

To train our network, we will first have to define a loss function, an optimizer and hyperparameters that control the training process:

* [`torch.optim.AdamW`](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html?highlight=adamw#torch.optim.AdamW) is used as an optimizer with default parameters except for the learning rate which is set to `lr=3e-4`.
* [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) is used as loss function. Note, that the softmax activation is applied during loss computation, as stated in the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
* The number of training epochs is 15


Train your multilayer perceptron network using the `Trainer` class and provide `trainLoader` as the DataLoader for training data and `valLoader` as the DataLoader for validation data.

The overall training should take about 20 seconds per epoch (**on a GPU**, depending on what GPU is assigned). Reported accuracies on the training (validation) data should be approx. 95% (81%) after 15 training epochs.   




In [ ]:
##### YOUR CODE GOES HERE ######
lr = 3e-4
model = network
param_optimizer = model.parameters()
lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(param_optimizer, lr=lr)

trainer = Trainer(model, lossFunction, optimizer, device)
trainer.train(trainLoader, valLoader, 15)
################################

### Visualize the behavior of the loss and accuracy (**add your code here**)

Using the data available in `trainer.metrics`, create the following two plots:
* Training loss and validation loss as a function of epochs.  
* Training accuracy and validation accuracy as a function of epochs.  

In [ ]:
##### YOUR CODE GOES HERE ######
trainLoss = trainer.metrics["epochTrainLoss"]
valLoss = trainer.metrics["epochValLoss"]
trainacc = trainer.metrics["epochTrainAccuracy"]
trainValacc = trainer.metrics["epochValAccuracy"]

# Number of epochs
epochs = range(1, len(trainLoss) + 1)

# Plotting training and validation loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, trainLoss, 'b', label='Training loss')
plt.plot(epochs, valLoss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, trainacc, 'b', label='Training accuracy')
plt.plot(epochs, trainValacc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
################################

### Run your network on some images to get predictions (**PROVIDED**)

In [ ]:
# Run some batches of unseen test data through the network and visualize its predictions
numBatches = 4
numRows = 8

 # Iterator through the test DataLoader
dataIter = iter(testLoader)

# for each batch
for batch in range(numBatches):

    # get images and ground truth labels
    images, labels = next(dataIter)

    # push to the device used
    images, labels = images.to(device), labels.to(device)

    # forward pass of the batch of images
    outputs = network(images)

    # find the index of the class with the largest output
    _, predictedLabels         = torch.max(outputs, 1)

    # print labels and outputs
    countCorrect = 0
    for i in range( batchSize // numRows ):
        for j in range((i*numRows), (i*numRows)+numRows):
            print(f'Image {j:2d} - Label: {classes[labels[j]]:5s} | Prediction: {classes[predictedLabels[j]]:5s}')
            if labels[j] == predictedLabels[j]:
                countCorrect = countCorrect + 1

    print(f"\n{(countCorrect / batchSize) * 100.0:.2f}% of test images correctly classified")

    # show images
    imshow(torchvision.utils.make_grid(images))

### Evaluate the performance on the unseen test data set.  (**add your code here**)

Use the proviced `Tester` class (see above) and test your trained network on the unseen test set available via `testLoader`. Your trained network should have approximately 80% accuracy (+- 2% depending on randomness during training)

In [ ]:
###### YOUR CODE GOES HERE ######
# Test the network on the final test set

# Test a neural network model
# create a tester
tester = Tester(model, device)
# test the model
tester.test(testLoader)

#################################